<a href="https://colab.research.google.com/github/AmanAryan007/AmanAryan007/blob/main/RaG_Model_For_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U  langchain llama-index langchain_experimental langchain_openai
!pip install sentence_transformers
!pip install faiss-cpu
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [11]:
pdf_path = '/content/AmanResume_07 (1).pdf'

In [13]:
import re
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PDF reader object
        pdf_reader = PdfReader(pdf_file)

        # Initialize an empty string to store the text
        full_text = ""

        # Iterate through each page and extract text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            full_text += page.extract_text()

    return full_text

def clean_text(text):
    # Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text)

    # Remove special characters except commas and periods
    text = re.sub(r'[^\w\s,.]', '', text)

    return text.strip()

def save_text_to_file(text, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(text)

# Replace 'input.pdf' with your PDF file path


# Extract text from PDF
extracted_text = extract_text_from_pdf(pdf_path)

# Clean extracted text
cleaned_text = clean_text(extracted_text)

# Replace 'output.txt' with your desired output text file path
output_file = 'document.txt'

# Save cleaned text to a text file
save_text_to_file(cleaned_text, output_file)

print(f"Cleaned text saved to {output_file}")


Cleaned text saved to document.txt


In [6]:
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
from typing import List
import faiss
import numpy as np

# Function to read text from a .txt document
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# File path to the .txt document
file_path = '/content/document.txt'

# Read the text from the document
text = read_text_from_file(file_path)

# Initialize the SentenceTransformer model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Use Proposition-Based Chunking

# Function to extract propositions using the summarization model
def get_propositions(text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    return summary[0]['summary_text'].split('. ')

# Splitting text into paragraphs and extracting propositions
paragraphs = text.split("\n\n")
text_propositions = []
for i, para in enumerate(paragraphs[:5]):
    propositions = get_propositions(para)
    text_propositions.extend(propositions)

print(f"You have {len(text_propositions)} propositions")
print(text_propositions[:10])

# Class to manage Agentic Chunking
class AgenticChunker:
    def __init__(self):
        self.propositions = []

    def add_propositions(self, propositions):
        self.propositions.extend(propositions)

    def pretty_print_chunks(self):
        chunks = self.get_chunks()
        return "\n\n".join(chunks)

    def get_chunks(self, chunk_size=5):
        chunks = [" ".join(self.propositions[i:i + chunk_size]) for i in range(0, len(self.propositions), chunk_size)]
        return chunks

# Using the AgenticChunker to chunk the text
ac = AgenticChunker()
ac.add_propositions(text_propositions)
print(ac.pretty_print_chunks())
chunks = ac.get_chunks()
print(chunks)

# Assuming Document is a class that takes page_content and metadata
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# Creating documents for each chunk
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]

# Initialize Faiss index for embeddings
index = faiss.IndexFlatL2(model.get_sentence_embedding_dimension())

# Track embeddings and corresponding documents
embeddings = []

# Adding documents to Faiss index
for doc in documents:
    embedding = model.encode(doc.page_content)
    index.add(np.array([embedding]))
    embeddings.append(embedding)

# Function to search in Faiss index
def search_faiss(query, k=2):
    query_embedding = model.encode(query)
    D, I = index.search(np.array([query_embedding]), k)
    return I

# Simulating Retrieval-Augmented Generation (RAG) with the chunks
def rag(documents):
    for doc in documents:
        print(f"Document from {doc.metadata['source']}: {doc.page_content}")

rag(documents)

# Initialize the generative model (BART)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
generator = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Function to generate response using retrieved documents
def generate_response(query, documents):
    retrieved_docs_indices = search_faiss(query, k=1)[0]
    retrieved_docs = [documents[i] for i in retrieved_docs_indices]
    context = " ".join([doc.page_content for doc in retrieved_docs])
    inputs = tokenizer.encode(query + " " + context, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = generator.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print()

# Example query and response
query = "What is the summary of resume?"
response = generate_response(query, documents)
print(f"Response: {response}")

# Example search
query = "What is the summary of resume?"
results = search_faiss(query, k=1)
print(f"Top {len(results[0])} results: {results}")

# Return the content of the query results
for i, idx in enumerate(results[0]):
    print(f"\nResult {i+1}:")
    print(f"Index: {idx}")
    print(f"Content: {documents[idx].page_content}")


You have 2 propositions
['Im a Python developer and AI specialist with a passion for machine learning, data analysis, and web development', 'Ive tackled a variety of challenging projects, like creating a tool to check driving license statuses, developing dashboards for trading analysis']
Im a Python developer and AI specialist with a passion for machine learning, data analysis, and web development Ive tackled a variety of challenging projects, like creating a tool to check driving license statuses, developing dashboards for trading analysis
['Im a Python developer and AI specialist with a passion for machine learning, data analysis, and web development Ive tackled a variety of challenging projects, like creating a tool to check driving license statuses, developing dashboards for trading analysis']
Document from local: Im a Python developer and AI specialist with a passion for machine learning, data analysis, and web development Ive tackled a variety of challenging projects, like creati